## 曲奇饼问题


In [1]:
from codeLib.thinkbayes import Pmf

在贝叶斯定理的语境下，可以很自然地使用一个 Pmf 映射每个假设和对应的概率。在曲奇饼问题里面， 该假设是 B1 和 B2。在 Python 中可以使用字符串来表示它们。

In [2]:
pmf = Pmf()
pmf.Set('Bowl1', 0.5)
pmf.Set('Bowl2', 0.5)

这一分布包含了对每个假设的先验概率，称为**先验分布**。

要更新基于新数据（拿到一块香草曲奇饼） 后的分布，我们将先验分别乘以对应的**似然度**。


从碗 1 拿到香草曲奇饼的可能性是 3/4，碗 2 的可能性是 1/2。

In [3]:
pmf.Mult('Bowl1', 0.75)
pmf.Mult('Bowl2', 0.5)

Mult 将给定假设的概率乘以已知的似然度。

更新后的分布还没有归一化，但由于这些假设互斥且构成了完全集合（意味着完全包含了所有可能假设），我们可以进行重新归一化如下：

In [4]:
pmf.Normalize()

0.625

In [5]:
print('假设是 碗1 的后验概率: %f %%'%(pmf.Prob('Bowl1')*100) )

假设是 碗1 的后验概率: 60.000000 %


## 贝叶斯框架

In [11]:
# 该类继承了Pmf类
class Cookie(Pmf):
    """A map from string bowl ID to probablity."""
    # mixes 是类变量,可以直接用类调用，或用实例对象调用
    mixes = { 'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
              'Bowl 2':dict(vanilla=0.5, chocolate=0.5)     }

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)  # 获取对应的相似度
            self.Mult(hypo, like)               # 计算后验概率
        self.Normalize()


    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type : 香草味/巧克力味
        hypo: string bowl ID
        """
        mix = self.mixes[hypo]
        like = mix[data]
        return like

In [12]:
print('Cookie 的父类是 : %s \n'%Cookie.__base__)


hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)



pmf.Update('vanilla')
for hypo, prob in pmf.Items():  # 返回pmf字典中dict的key和value
    print(hypo, prob)

Cookie 的父类是 : <class 'codeLib.thinkbayes.Pmf'> 

Bowl 1 0.6000000000000001
Bowl 2 0.4


## Monty Hall 难题


In [13]:
class Monty(Pmf):
    """Map from string location of car to probability"""

    def __init__(self, hypos):
        """Initialize the distribution.

        hypos: sequence of hypotheses
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):  # 设置Monty要打开那个门
        """Updates each hypothesis based on the data.

        data: any representation of the data
        """
        for hypo in self.Values():  # dict.key
            like = self.Likelihood(data, hypo)  # [data]Monty打开的门，[hypo]奖品所在的门
            self.Mult(hypo, like)
        self.Normalize()

    def Likelihood(self, data, hypo):
        """Compute the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is  [奖品所在的门]
        data: string name of the door Monty opened        [Monty打开的门]
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [14]:
hypos = 'ABC'
pmf = Monty(hypos)

data = 'B'
pmf.Update(data)

for hypo, prob in sorted(pmf.Items()):
    print(hypo, prob)

A 0.3333333333333333
B 0.0
C 0.6666666666666666


## 封装框架

现在，我们看看框架的哪些元素是相同的，这样我们就可以把它们封装进一个 Suite对象，即一个提供__init__， Update 和 Print 方法的 pmf 对象。

Suite 的实现在 thinkbayes.py 中。要使用 Suite 对象，你应当编写一个继承自 Suite的类，并自行提供 Likelihood 方法的实现。例如，这是一个以蒙蒂大厅问题改写的使用 Suite 的方案。

In [18]:
from codeLib.thinkbayes import Suite

class Monty(Suite):
    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1


In [20]:
# main() 主函数
suite = Monty('ABC')
suite.Update('B')
suite.Print()

A 0.3333333333333333
B 0.0
C 0.6666666666666666


## M&M 豆问题

In [22]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [24]:
#  main 主函数
suite = M_and_M('AB')
print('先验概率:')
suite.Print()

# 黄色 M＆M 豆的袋子称为袋 1，绿色的袋2
suite.Update(('bag1', 'yellow'))  
suite.Update(('bag2', 'green'))
print()

# A假设: bag1是94年的，bag2是96年的
# B假设: bag1是96年的，bag2是94年的
print('后验概率:')
suite.Print()

先验概率:
A 0.5
B 0.5

后验概率:
A 0.7407407407407407
B 0.2592592592592592
